## Dependencies

In [1]:
# Load the sentence transformer model
#from resume_parser import resumeparse
#from transformers import BertModel, BertTokenizer
#import torch
import glob
import os
import pinecone
import numpy as np
import json
#from sentence_transformers import SentenceTransformer
from pinecone import Index
import tkinter as tk
from tkinter import scrolledtext
from tkinter import filedialog
from tkinter import messagebox
from ttkthemes import ThemedTk
from tkinter.font import Font
import sys
from PyPDF2 import PdfFileReader
from docx import Document
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
import textract
from langchain.embeddings.openai import OpenAIEmbeddings
import re
from tkinter import scrolledtext, messagebox
import urllib
import urllib.parse
from pymongo.mongo_client import MongoClient
import unidecode
from tkinter import BooleanVar
import time
import openai

/opt/anaconda3/envs/py10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import pinecone
import os
import numpy as np
import json
import glob

## Pinecone and MongoDB initialization

In [3]:

# pinecone vector database
pinecone.init(api_key="bc6d7eb3-c8aa-4537-acaa-f27f9c417c9f", environment= 'us-west4-gcp')
filename1 = 'resumes.json'
pinecone.init(api_key="bc6d7eb3-c8aa-4537-acaa-f27f9c417c9f", environment= 'us-west4-gcp')
index_name = "cloudresumes"
index = pinecone.Index(index_name=index_name)


# openai embedding
model_name = 'gpt-4'
# Set up your own OpenAI private key

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)


# MongoDB
# Set up with your own account
client = MongoClient(uri)
db = client["resumesamples"]
collection = db["re"]           

## Process Resuems

In [4]:
def process_pdf(file_path, file_name):
    with open(file_path, "rb") as file:
        pdf = PdfReader(file)
        text = ' '.join([pdf.pages[i].extract_text() for i in range(len(pdf.pages))])

    return extract_info(text, file_name)

def process_docx(file_path, file_name):
    doc = Document(file_path)
    text = " ".join([paragraph.text for paragraph in doc.paragraphs])

    return extract_info(text, file_name)

def process_html(file_path, file_name):
    with open(file_path, "r", encoding='iso-8859-1') as file:
        soup = BeautifulSoup(file, 'html.parser')
        text = soup.get_text()

    return extract_info(text, file_name)

def process_doc(file_path, file_name):
    text = textract.process(file_path).decode('utf8')

    return extract_info(text, file_name)

def process_txt(file_path, file_name):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    return extract_info(text, file_name)

def extract_info(text, file_name):
    return {'id': file_name, 'text': text}

def read_resume_files(resume_folder):
    """
    read all files from a folder and return a dictionary of json format
    """
    resume_files = os.listdir(resume_folder)
    print(f"there are {len(resume_files)} resumes")

    json_resumes = []
    # Iterate over all files
    for file in resume_files:
        try:
            file_path = os.path.join(resume_folder, file)
            
            if file.endswith('.pdf'):
                json_resumes.append(process_pdf(file_path, file))
            elif file.endswith('.docx'):
                json_resumes.append(process_docx(file_path, file))
            elif file.endswith('.html'):
                json_resumes.append(process_html(file_path, file))
            elif file.endswith('.doc'):
                json_resumes.append(process_doc(file_path, file))
            elif file.endswith('.txt'):
                json_resumes.append(process_txt(file_path, file))
            else:
                print(f"Unsupported file type: {file}")
        except Exception as e:
            print(f"Failed to process {file} due to {e}" )

    return json_resumes

def read_single_resume_file(filepath):
    """
    read a single resume file and return a dictionary of ids and text
    """
    try:
        file_name = os.path.basename(filepath)

        if filepath.endswith('.pdf'):
            return process_pdf(filepath, file_name)
        elif filepath.endswith('.docx'):
            return process_docx(filepath, file_name)
        elif filepath.endswith('.html'):
            return process_html(filepath, file_name)
        elif filepath.endswith('.doc'):
            return process_doc(filepath, file_name)
        elif filepath.endswith('.txt'):
            return process_txt(filepath, file_name)
        else:
            print(f"Unsupported file type: {file_name}")
            return None
    except Exception as e:
        print(f"Failed to process {file_name} due to {e}")
        return None


# Use OpenAi's embedding

### Skip the following lines, create new Pinecone index

In [5]:
# #### Do not run this line, already exsists
# # Create a new Pinecone index
# pinecone.create_index(name=index_name, metric="cosine", shards=1, dimension = 1536)
# # Connect to the index
# index = pinecone.Index(index_name=index_name)

### start next lines, init GPT

## DB functions

In [6]:
def get_top_matches(index, embed, job_description, top_k):
    """get top matches using Pinecone's vector search, the model is specified by embed"""
    # Convert job description into embeddings
    query_vector = embed.embed_documents([job_description])
    query_vector = query_vector[0] # Select the first (and only) embedding
    #print(query_vector)

    # Query Pinecone for similar vectors
    query_result = index.query(queries=[query_vector], top_k=top_k, namespace=index_name)
    #print(len(query_vector))

    # Get IDs of top_k similar vectors
    print(query_result)

    return query_result


In [7]:
def fetch_all_resumes_from_db():
    """
    fetch all resumes from MongoDB
    return: a list of dictionaries
    """
    cursor = collection.find({}, {'_id': 0})  # Exclude the '_id' field.

    json_resumes = [doc for doc in cursor]

    return json_resumes

def fetch_resume_by_id(resume_id):
    """Fetch a single resume by ID from MongoDB instead of the local JSON file"""
    # Fetch the resume from MongoDB
    resume = collection.find_one({"id": resume_id}, {'_id': 0})
    return str(resume) if resume else None

In [8]:
def clean_id(id_string):
    return ''.join([i if ord(i) < 128 else '_' for i in id_string])

def upload_resumes():

    # Open directory dialog and get folder path
    resume_folder = filedialog.askdirectory()
    print(f"Selected folder: {resume_folder}")

    # Connect to MongoDB
    # password required
    # uri required
    client = MongoClient(uri)
    db = client["resumesamples"]
    collection = db["re"]
    print(f"MongoDB connected")
    
    for dirpath, dirs, files in os.walk(resume_folder):
        # Process the resumes
        for filename in files:
            fullpath = os.path.join(dirpath, filename)
            try:
                resume = read_single_resume_file(fullpath)

                # Clean the resume ID before processing
                ascii_id = clean_id(resume["id"])
                resume["id"] = ascii_id

                # Check if the resume already exists in MongoDB and Pinecone
                existing_document = collection.find_one({"id": resume["id"], "pinecone_id": {"$exists": True}})
                if existing_document is not None:
                    print(f'Skipped document with id: {resume["id"]} as it already exists in MongoDB and Pinecone.')
                    continue

                # Write each resume to MongoDB
                existing_document = collection.find_one({"id": resume["id"]})
                if existing_document is not None:
                    # Update the existing document
                    result = collection.update_one({"id": resume["id"]}, {"$set": resume})
                    print(f'Updated the document with id: {resume["id"]}')
                else:
                    # Insert a new document
                    result = collection.insert_one(resume)
                    print(f'Added a new document with id: {resume["id"]}')

            except Exception as e:
                print(f"Failed to process file: {fullpath}, Error: {e}")
                continue  # this will skip the rest of this loop and go to the next file

    # Feedback to user
    print(f"MongoDB Updated for folder: {resume_folder}")
    
    #update Pinecone
    update_pinecone()


def insert_into_pinecone(index, embed, json_resumes):    
    inserted_ids = []
    for resume in json_resumes:
        try:
            
            all_fields_text = " ".join([str(field) + ':' + str(resume.get(field, "")) + ";" for field in resume.keys()-'id'])
            # Transform the text into embeddings using OpenAI
            embeddings = embed.embed_documents([all_fields_text])

            # Convert embeddings to a list as Pinecone expects a list or numpy array
            embeddings = embeddings[0]  # Select the first element because model.encode returns a list of embeddings

            # Convert the resume ID to ASCII-compatible format using the clean_id function
            ascii_id = clean_id(resume['id'])

            # upsert (update/insert) the vector into Pinecone using the cleaned ID
            upsert_response = index.upsert(vectors=[{'id': ascii_id, 'values': embeddings}], namespace=index_name)

            print(f"Finished inserting {ascii_id} into Pinecone")
            inserted_ids.append(ascii_id)

        except Exception as e:
            print(f"Failed to process {ascii_id}, Error: {e}")
            continue

    return inserted_ids

def update_pinecone():
    # ... (existing code) ...
    # password
    # uri
    client = MongoClient(uri)
    db = client["resumesamples"]
    collection = db["re"]
    # pinecone init
    index = pinecone.Index(index_name=index_name)

    # Only select the resumes with IDs not in pinecone_ids
    all_resumes = list(collection.find({"pinecone_id": {"$exists": False}}))
    json_resumes = all_resumes

    # Update the pinecone_ids
    new_ids = insert_into_pinecone(index, embed, json_resumes)
    for id in new_ids:
        # Update MongoDB with new Pinecone IDs
        collection.update_one({"id": id}, {"$set": {"pinecone_id": id}})

    print("Updated Pinecone.")

In [9]:
def flush_databases():
    # ... (existing code) ...

    # Flush MongoDB
    collection.delete_many({})

    print("MongoDB has been flushed.")

    # Connect to Pinecone
    # pinecone init
    index = pinecone.Index(index_name=index_name)

    # Flush Pinecone
    index.delete(deleteAll='true', namespace=index_name)
    
    print("Pinecone has been flushed.")


## Layer2 Boolean Search

In [10]:
def parse_query(query):
    """ boolean search parse query to tokens"""
    query = query.lower()
    tokens = re.findall(r'\([^)]*\)|"[^"]*"|\S+', query)
    return [token.strip('"') for token in tokens]

def boolean_search(resume_text, query):
    """ implement boolean search return True/False"""
    
    if not resume_text:
        return False
    
    resume_text = resume_text.lower()

    tokens = parse_query(query)

    def process_keyword(keyword, negation=False):
        if '*' in keyword:  # handle wildcard searches
            keyword = keyword.replace('*', '.*')  # convert * to regex pattern
            keyword_present = bool(re.search(keyword, resume_text))
        else:
            keyword_present = keyword in resume_text

        if negation:
            print(f"Keyword '{keyword}' is not in text: {not keyword_present}")  # Debug print statement
            return not keyword_present
        else:
            print(f"Keyword '{keyword}' is in text: {keyword_present}")  # Debug print statement
            return keyword_present

    def evaluate_expression(tokens):
        if not tokens:
            return False

        keyword = tokens.pop(0)
        print(f"Processing keyword: {keyword}")  # Debug print statement

        negation = False
        if keyword == "not":
            negation = True
            keyword = tokens.pop(0)

        if keyword.startswith("("):
            result = evaluate_expression(parse_query(keyword.strip("()")))
        elif keyword:
            result = process_keyword(keyword, negation)
        
        while tokens:
            operator = tokens.pop(0)
            print(f"Processing operator: {operator}")  # Debug print statement

            if operator == ")":
                return result

            next_keyword = tokens.pop(0) if tokens else None
            print(f"Processing next keyword: {next_keyword}")  # Debug print statement

            if next_keyword and next_keyword.startswith("("):
                next_result = evaluate_expression(parse_query(next_keyword.strip("()")))
            elif next_keyword:
                next_result = process_keyword(next_keyword)

            print(f"Intermediate result before applying operator: {result}")  # Debug print statement
            if operator == 'and':
                result = result and next_result
            elif operator == 'or':
                result = result or next_result
            elif operator == 'not':
                result = result and not next_result
            print(f"Intermediate result after applying operator: {result}")  # Debug print statement

        return result

    return evaluate_expression(tokens)

## Layer3 AI check

In [11]:
def AI_check(resume_text, job_description, strict_requirements=None):
    # OpenAI key
    prompt = """
    "Answer Yes or No: 
    You are a knowledgeable hiring manager. Your task is to review the given resume and check if the candidate satisfies the job description or potentially qualifies the job. 
    Please respond with a one or two sentences reason 'Yes, He/She is. Because...' if the candidate satisfies the job description, and 'No, because...' otherwise. 
    Don't be strict on it, but you can infer from his resume on whether he will be able to do this job. For example, a Phd in computer science who know nothing in finance might fit well on quant developer even if the requirements says CFA preferred.
    Be very tolerative on other fields even if he/she lacks of some skills on resume.
    """
    while True:
        try:
            # Create the messages for the conversation with the model
            messages = [
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Here is the job description:\n\n{job_description}\n\nAnd here is the resume:\n\n{resume_text}"} 
            ]
            
            if strict_requirements is not None and strict_requirements != "":
                messages.append({"role": "user", "content": f"And here are the strict requirements you need to check:\n\n{strict_requirements}"})

            # Request a completion from the OpenAI API
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k",
                messages=messages,
            )

            # Extract the model's response
            model_response = response['choices'][0]['message']['content']

            # Strictly match the response to 'Yes' or 'No'
            if "yes" in model_response.strip().lower():
                print(model_response)
                return True
            elif "no" in model_response.strip().lower():
                print(model_response)
                return False
            else:
                raise ValueError(f"Unexpected response from AI: {model_response}")
        
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Retrying in 5 seconds...")
            time.sleep(5)

## Search function

In [12]:
def process_resume(text):
    # OpenAI key
    prompt = """Strictly Structure the candidate's raw resumes in the following format using both resume content and your knowledge externally in human readable way.
     "Contact information": {
        "Name": "<their name>",
        "Email": "<their email>",
        "Phone": "<their phone>"
        },
    "Summary": In no more than 10 sentences, summary of candidate's current role, what's his/her specializations.
    "Level": "<approximate how many years of working experience does the candidate have, if not easy to calculate, just classify into new graduate, entry level, mid level, and senior level along with your approximation of years"
    "Skills": "<List of their techinical skills, separate by comma>",
    "Educations": "<Summary of candidate's education experience, like schools, degrees, and majors>"
    """

    # Define the series of messages
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": text}
    ]

    # Request a completion
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
    )

    # Return the model's message from the response
    return response['choices'][0]['message']['content']

def on_result_select(event):
    try:
        # Get selected line index
        selected_index = result_listbox.curselection()[0]
        selected_text = result_listbox.get(selected_index)

        # Retrieve the full resume using the ID from selected_text, and generate a summary
        try:
            resume_text = fetch_resume_by_id(selected_text)  # Use the function you already have to get resume text by id
        except Exception as e:
            print(f"Error fetching resume with id {selected_text}: {str(e)}")
            tk.messagebox.showerror("Error", f"Error fetching resume with id {selected_text}: {str(e)}")
            return

        try:
            resume_summary = process_resume(resume_text)
        except Exception as e:
            print(f"API overload while processing resume with id {selected_text}: {str(e)}")
            time.sleep(5)  # Retry after 5 seconds
            try:
                resume_summary = process_resume(resume_text)
            except Exception as e:
                print(f"Failed to process resume with id {selected_text} after retrying: {str(e)}")
                tk.messagebox.showerror("Error", f"Failed to process resume with id {selected_text} after retrying: {str(e)}")
                return

        # Display the summary
        tk.messagebox.showinfo("Resume Summary", resume_summary)
    except Exception as e:
        print(f"Unexpected error in on_result_select: {str(e)}")
        tk.messagebox.showerror("Error", f"Unexpected error: {str(e)}")



In [13]:
# Here's a revised version of your `search_resumes` function.
# I've assumed that the 'id' field in the match object is the resume id.

def search_resumes(top_k):
    global result_listbox  # Declare result_listbox as global
    if result_listbox is None:
        result_listbox = tk.Listbox(root)
        result_listbox.grid(row=10, column=0, padx=10, pady=10, sticky="ew")
    else:
        result_listbox.delete(0, "end")

    # Fetch all resumes from MongoDB instead of the local JSON file
    json_resumes = fetch_all_resumes_from_db()
    print("loaded json resumes")
    
    job_description = job_description_entry.get("1.0", "end-1c")  
    keyword = keyword_entry.get("1.0", "end-1c")

    print(f"Job Description: {job_description}")  
    print(f"Keyword: {keyword}")  

    try:
        top_matches = get_top_matches(index, embed, job_description, top_k=top_k)
        print(f"Top Matches: {top_matches}")  

        matches = top_matches['results'][0]['matches']
        filtered_matches = []

        for match in matches:
            resume_id = match['id']

            # Fetch the resume text from MongoDB using the ID
            resume_text = fetch_resume_by_id(resume_id)

            if keyword and not boolean_search(resume_text, keyword):
                continue  # Skip this match if it doesn't pass the boolean search

            filtered_matches.append(resume_id)
        
        if AI_checking_checkbox.get():
            strict_requirements = strict_requirements_entry.get("1.0", "end-1c")
            for match in filtered_matches:
                resume_text = fetch_resume_by_id(match)
                print(f"AI checking for {match}")
                if AI_check(resume_text, job_description, strict_requirements):
                    result_listbox.insert("end", match)
        else:
            for match in filtered_matches:
                result_listbox.insert("end", match)

    except Exception as e:
        print(f"Error: {e}")  
        messagebox.showerror("Error", str(e))

## Delete function

In [14]:
# Delete entry in Pinecone
def pinecone_delete(id_to_delete):
    """
    Delete entries from a Pinecone index.

    """

    # Delete the entries
    delete_op = index.delete(id_to_delete, namespace=index_name)
    print(delete_op)
    # Make sure the operation completes before continuing
    #delete_op.result()
    
    print(id_to_delete, "deleted from Pinecone")

    
# Delete entry in MongoDB
def mongo_delete(id_to_delete):
    """
    Delete specific document from a MongoDB collection.
    """
    
    collection.delete_one({ "id": str(id_to_delete) })
    
    print(id_to_delete, "deleted from MongoDB")
    
def delete_doc():
    id_to_delete = delete_entry.get("1.0", "end-1c")
    print(id_to_delete)
    
    if id_to_delete == "":
        return
    
    # Ask for user's confirmation to prevent accidental delete
    confirm = tk.messagebox.askquestion ("Confirmation", f"Are you sure you want to delete ID: {id_to_delete}?" ,icon = 'warning')
    if confirm == "yes":    
        print(f"Deleting ID: {id_to_delete}")
        pinecone_delete(id_to_delete)
        mongo_delete(id_to_delete)
    else:
        return

## GUI implementation

In [15]:
if __name__ == "__main__":
    class CreateToolTip:
        def __init__(self, widget, text='widget info'):
            self.widget = widget
            self.text = text
            self.widget.bind("<Enter>", self.enter)
            self.widget.bind("<Leave>", self.close)

        def enter(self, event=None):
            x, y, _, _ = self.widget.bbox("insert")
            x += self.widget.winfo_rootx() + 25
            y += self.widget.winfo_rooty() + 20
            self.tw = tk.Toplevel(self.widget)
            self.tw.wm_overrideredirect(True)
            self.tw.wm_geometry(f"+{x}+{y}")
            label = tk.Label(self.tw, text=self.text, background="#ffffff", relief='solid', borderwidth=1,
                             wraplength=180)
            label.pack(ipadx=1)

        def close(self, event=None):
            if self.tw:
                self.tw.destroy()

    class PrintLogger:  # create file like object
        def __init__(self, textbox):  # pass reference to text widget
            self.textbox = textbox  # keep ref

        def write(self, text):
            self.textbox.insert(tk.END, text)  # write text to textbox
            self.textbox.see(tk.END)
            self.textbox.update_idletasks()  # update GUI

            
    def flush_databases_with_confirmation():
        MsgBox = tk.messagebox.askquestion ('Flush Database','Are you sure you want to flush the database?',icon = 'warning')
        if MsgBox == 'yes':
            flush_databases()

    def on_closing():
        # Reset standard output to default
        sys.stdout = sys.__stdout__

        # Any necessary cleanup here
        # Then destroy window
        root.destroy()
        # And finish the event loop
        root.quit()
        
        
    root = ThemedTk(theme="plastik")
    root.title("Resume Search Engine")
    
    job_description_label = tk.Label(root, text="Job Description:")
    job_description_entry = scrolledtext.ScrolledText(root, width=80, height=10)
    CreateToolTip(job_description_entry, "(Optional) Describe ideal candidates or paste job descriptions in natural English or paste Job description here.")
    
    keyword_label = tk.Label(root, text="Keyword:")
    keyword_entry = scrolledtext.ScrolledText(root, width=100, height=2) 
    CreateToolTip(keyword_entry, "(Optional) Boolean Search, for example: (Python and C++ or quant*). please check documentations on how to use boolean search ")
    
    top_k_label = tk.Label(root, text="Searching Scope:")
    CreateToolTip(top_k_label, "The amount of resumes in the Database used for searching, Larger Scope -> preciese but slow")
    top_k_entry = tk.Entry(root)
    top_k_entry.insert(0, "100")  # Setting default value as 200
    CreateToolTip(top_k_entry, "Larger the scope -> the better the reuslts but will be SLOW! Recommand not exceed 500")
    
    strict_requirements_label = tk.Label(root, text="Strict Requirements:")
    CreateToolTip(strict_requirements_label, "Enable AI checking to use, AI will check the requirements strictly")
    strict_requirements_entry = scrolledtext.ScrolledText(root, width=100, height=2)
    CreateToolTip(strict_requirements_entry, "(Optional, select enable AI checking box to use) Specify any strict requirements for the job. AI will strictly look at those requirements. e.g. This person must have a phd in Computer Science and know ETL, AWS, and SQL.")
    
    # Delete label
    delete_label = tk.Label(root, text="Delete resumes: ")
    # Delete entry
    delete_entry = scrolledtext.ScrolledText(root, width=30, height=2)
    CreateToolTip(delete_label, "Please use this function carefully")
    
    AI_checking_checkbox = BooleanVar()
    AI_checking_checkbox.set(False)  # Unchecked by default
    AI_checking_checkbutton = tk.Checkbutton(root, text="Enable AI checking", variable=AI_checking_checkbox)
    CreateToolTip(AI_checking_checkbutton, "(Optional) Check this box to enable AI checking of resumes. AI will check resumes one by one accoridng to JD. Precise but Slow!")
    
    search_button = tk.Button(root, text="Search", command=lambda: search_resumes(int(top_k_entry.get())), font=Font(size=20))
    CreateToolTip(search_button, "click here to search resumes")
    
    upload_button = tk.Button(root, text="Upload Resumes", command=upload_resumes)
    CreateToolTip(upload_button, "Select a folder, will automatically upload all resumes in the folder and its subdirectories")
    
    flush_button = tk.Button(root, text="Flush Database", command=flush_databases_with_confirmation)
    CreateToolTip(flush_button, "Warning! This button will Permanently delete all data from the database.")
    
    # Delete button
    delete_button = tk.Button(root, text="Delete", command=delete_doc, cursor="hand2")
    CreateToolTip(delete_button, "Warning! This button will cause permanent loss of information.")
    
    result_label = tk.Label(root, text="Results:")
    
    logger_box = scrolledtext.ScrolledText(root, width=80, height=10)
    CreateToolTip(logger_box, "Console messages here, You will be able to see the reasoning given by AI")
    pl = PrintLogger(logger_box)
    sys.stdout = pl

    job_description_label.grid(row=0, column=0, sticky="w")
    job_description_entry.grid(row=1, column=0, padx=10, pady=10, sticky="ew")

    keyword_label.grid(row=2, column=0, sticky="w")
    keyword_entry.grid(row=3, column=0, padx=10, pady=10, sticky="ew")

    top_k_label.grid(row=4, column=0, sticky="w")
    top_k_entry.grid(row=4, column=1, sticky="ew")

    strict_requirements_label.grid(row=5, column=0, sticky="w")
    strict_requirements_entry.grid(row=5, column=1, sticky="ew")
    
    # Delete label entry grid
    delete_label.grid(row=0, column=1, sticky="w")
    delete_entry.grid(row=1, column=1, padx=0, pady=0, sticky="nw")
    
    
    
    
    AI_checking_checkbutton.grid(row=6, column=0, sticky="w")

    search_button.grid(row=7, column=0, padx=10, pady=10, sticky="ew")
    upload_button.grid(row=8, column=0, padx=10, pady=10, sticky="ew")
    # resize delete button
    delete_button.grid(row=1, column=1, padx=200, pady=0, sticky="ne")

    result_label.grid(row=9, column=0, padx=10, pady=10, sticky="w")
    
    result_listbox = tk.Listbox(root)
    result_listbox.grid(row=10, column=0, padx=10, pady=10, sticky="ew")
    result_listbox.bind('<Double-1>', on_result_select)

    logger_box.grid(row=11, column=0, padx=10, pady=10, sticky="ew")
    flush_button.grid(row=12, column=0, padx=10, pady=10, sticky="e")
    
    root.grid_columnconfigure(0, weight=1)
    root.grid_rowconfigure((1, 3, 10, 11), weight=1)

    root.protocol("WM_DELETE_WINDOW", on_closing)

    root.mainloop()




2024-03-19 18:40:47.313 python[40629:74362601] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
